# Day 11: Dumbo Octopus
There are 100 octopuses arranged neatly in a 10 by 10 grid. Each octopus slowly gains energy over time and flashes brightly for a moment when its energy is full.
The energy level of each octopus is a value between 0 and 9.
During a single step, the following occurs:

First, the energy level of each octopus increases by 1.
Then, any octopus with an energy level greater than 9 flashes. This increases the energy level of all adjacent octopuses by 1, including octopuses that are diagonally adjacent. If this causes an octopus to have an energy level greater than 9, it also flashes. This process continues as long as new octopuses keep having their energy level increased beyond 9. (An octopus can only flash at most once per step.)
Finally, any octopus that flashed during this step has its energy level set to 0, as it used all of its energy to flash.
Adjacent flashes can cause an octopus to flash on a step even if it begins that step with very little energy.
How many total flashes are there after 100 steps?

In [ ]:
let inline stepGrid (grid:seq<seq<int*int>>) =
    let inline bump (b,carry) = match b + 1 with 10 -> (0, carry + 1) | a -> (a,carry)
    seq {
        for y = 0 to 9 do
            yield grid |> Seq.item y |> Seq.map bump
    }
    // |> Array2D.mapi bump 

let rec runSteps max acc grid = 
    if acc = max then grid
    else grid |> stepGrid |> runSteps max (acc+1)

/// Expands a string of numbers into a sequence of floats less than 1.0
let inline expand (str:string) =
    let rec decompose (buff) (l:Int64)  =
        let tail  = l % 10L |> float |> (fun fl -> fl / 10.0)
        let head :Int64 = l / 10L
        let b = seq { yield tail; yield! buff }
        if l < 10 then 
            b
        else 
            decompose b head

    str |> int64 |> decompose Seq.empty

// let answer = 
System.IO.File.ReadAllLines("../Data/Day11.txt")
|> Array.map expand
    // |> Array.map (fun line -> Seq.map (fun c -> double (int c - int '0')) line)
// |> runSteps 0 0

// answer

index,value
0,"[ 0.4, 0.1, 0.1, 0.2, 0.2, 0.5, 0.6, 0.3, 0.7, 0.2 ]"
1,"[ 0.3, 0.1, 0.4, 0.3, 0.2, 0.5, 0.3, 0.7, 0.1, 0.2 ]"
2,"[ 0.4, 0.5, 0.1, 0.6, 0.8, 0.4, 0.8, 0.6, 0.3, 0.1 ]"
3,"[ 0.3, 0.7, 0.8, 0.3, 0.4, 0.7, 0.7, 0.1, 0.3, 0.7 ]"
4,"[ 0.3, 0.7, 0.4, 0.6, 0.7, 0.2, 0.3, 0.5, 0.8, 0.2 ]"
5,"[ 0.5, 0.8, 0.6, 0.1, 0.3, 0.5, 0.8, 0.8, 0.8, 0.4 ]"
6,"[ 0.4, 0.8, 0.4, 0.3, 0.3, 0.5, 0.1, 0.7, 0.7, 0.4 ]"
7,"[ 0.2, 0.3, 0.1, 0.6, 0.4, 0.4, 0.7, 0.6, 0.2, 0.1 ]"
8,"[ 0.6, 0.6, 0.4, 0.3, 0.8, 0.1, 0.7, 0.7, 0.4, 0.5 ]"
9,"[ 0.6, 0.3, 0.6, 0.6, 0.8, 0.1, 0.5, 0.8, 0.6, 0.8 ]"


## Thoughts
After applying brute force to 10 iterations of just flashing individuals with no spillover, we can determine that the flash rate for each squid is equal to the number of steps / 10. In other words if a squid were alone, over 120 steps a single squid would flash 12 times and after 120 steps, the squid's energy level would be equal to where it started as 120 is a multiple of 10.

Given the goal is to count the number of flashes, we know the answer with ***at least*** be equal to the number of squids * (steps / 10). I.E. 1 squid over 120 steps would be 12 flashes; 20 squids over 109 steps would be 200 flashes.

We also know that a squid cannot flash more than once each step which means that theoretically, if each squid somehow managed to flash every step, then they'd ***at most*** contribute a total number of flashes equal to the number of steps. I.E. we can use the step counter as max/clamp per step to determine whether a squid has flashed per step.

To solve this, we'll decompose each number in each string into a floating point number less than 1.0. Each step we'll increment by .1 which will allow our whole number to keep the state of our flashes. At the same time we'll clamp our whole number to never be higher than our step. This way when we hit our max steps, we'll just sum our whole numbers for the answer.